In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
state_dict = dict(zip(
    ["Kasaragod","Kannur","Trivandrum","Kollam","Pathanamthitta","Alappuzha","Idukki",
     "Ernakulam","Thrissur","Palakkad","Malappuram","Kozhikkode","Wayanad", "Kottayam"],      
    ["KGD", "KNR", "TVM", "KLM", "PTA", "ALP", "IDK", 
     "EKM", "TSR", "PKD", "MPM", "KKD", "WND", "KTM"]
))

state_dict_full = list(state_dict.keys())

In [18]:
def get_data(url):
          
    url =url
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    col=[]   # empty list
    i=0
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
      i+=1
      name=t.text_content()
      #print (i,name)
      col.append((name,[]))

      #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 10, the //tr data is not from our table 
        if len(T)!=5:
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
                #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    df.columns=df.columns.str.replace('.','')
    df.drop_duplicates() 
    #print(df)
    return df


In [23]:
Dataframe_home_hospital = get_data('https://dashboard.kerala.gov.in/quarantine-view-public.php')
Dataframe_home_hospital.head()

,Dist,Total,Hospital,Home,Hosp Today
0,Date,Total,Hospital,Home,Hosp. Today
1,Dist.,Total,Hospital,Home,Hosp. Today
2,PKD,10417,31,10386,9
3,KKD,10012,25,9987,6
4,WND,6877,4,6873,3


In [39]:
# Getting Home_isolation data
home_import= dict(zip(Dataframe_home_hospital.Dist, Dataframe_home_hospital.Home))
home_isolation = [home_import[x] for x in state_dict.values()]

# Getting Hospital data
hospital_import= dict(zip(Dataframe_home_hospital.Dist, Dataframe_home_hospital.Hospital))
hospital_isolation = [hospital_import[x] for x in state_dict.values()]

In [41]:
Dataframe_active = get_data('https://dashboard.kerala.gov.in/dailyreporting-view-public.php')
Dataframe_active.head()

,District,Reported,Cured,Active,Death
0,District,Reported,Cured,Active,Death
1,KGD,168,115,53,0
2,KNR,85,38,47,0
3,EKM,24,20,3,1
4,KKD,20,9,11,0


In [43]:
# Getting Hospital data
active_import= dict(zip(Dataframe_active.District, Dataframe_active.Active))
active = [active_import[x] for x in state_dict.values()]

In [44]:
d = {'State':state_dict_full,'HomeIsolation':home_isolation, 'HospitalIsolation':hospital_isolation, 'ActiveCases':active}
data_fetched = pd.DataFrame(d)
data_fetched

,State,HomeIsolation,HospitalIsolation,ActiveCases
0,Kasaragod,5740,117,53
1,Kannur,5823,111,47
2,Trivandrum,1698,113,2
3,Kollam,3257,9,5
4,Pathanamthitta,2844,14,6
5,Alappuzha,4767,6,2
6,Idukki,2445,4,0
7,Ernakulam,358,20,3
8,Thrissur,4552,10,1
9,Palakkad,10386,31,2


In [45]:
data_fetched.to_excel("DataFetch.xlsx",
             sheet_name='Sheet_name_1')  